**классификатор изображений LeNet**

Евгений Борисов borisov.e@solarl.ru

---

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

from tensorflow.keras.datasets import mnist
from tensorflow.keras import utils 

In [5]:
np.random.seed(1671)

In [32]:
# data: shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [33]:
# # tensorflow
# from tensorflow.keras import backend as K
# K.set_image_dim_ordering("th")

In [ ]:
# consider them as float and normalize

X_train /= 255.0
X_test /= 255.0

X_train = X_train[:, :, :, np.newaxis].astype('float32')
X_test = X_test[:, :, :, np.newaxis].astype('float32')

In [35]:
# convert class vectors to binary class matrices
y_train = utils.to_categorical(y_train, NB_CLASSES)
y_test = utils.to_categorical(y_test, NB_CLASSES)

In [36]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 28, 28, 1), (60000, 10), (10000, 28, 28, 1), (10000, 10))

In [37]:
# network and training
NB_EPOCH = 2
BATCH_SIZE = 128
VERBOSE = 1
VALIDATION_SPLIT=0.2

IMG_ROWS, IMG_COLS = 28, 28 # input image dimensions
NB_CLASSES = 10  # number of outputs = number of digits
INPUT_SHAPE = ( IMG_ROWS, IMG_COLS, 1)

In [38]:
# initialize the optimizer and model
# model = LeNet.build(input_shape=INPUT_SHAPE, classes=NB_CLASSES)

model = Sequential()
# CONV => RELU => POOL
model.add(Conv2D(20, kernel_size=5, padding="same", input_shape=INPUT_SHAPE))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# CONV => RELU => POOL
model.add(Conv2D(50, kernel_size=5, padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# Flatten => RELU layers
model.add(Flatten())
model.add(Dense(500))
model.add(Activation("relu"))
# a softmax classifier
model.add(Dense(NB_CLASSES))
model.add(Activation("softmax"))

In [ ]:
model.summary()

In [39]:
model.compile(loss="categorical_crossentropy", optimizer='rmsprop',metrics=["accuracy"])

In [40]:
%%time

history = model.fit(X_train, y_train, 
                    batch_size=BATCH_SIZE, 
                    epochs=NB_EPOCH, 
                    verbose=VERBOSE, 
                    validation_split=VALIDATION_SPLIT)

Train on 48000 samples, validate on 12000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
48000/48000 [==============================] - 222s 5ms/sample - loss: 0.1798 - acc: 0.9429 - val_loss: 0.0529 - val_acc: 0.9845
Epoch 2/20
 3200/48000 [=>............................] - ETA: 3:13 - loss: 0.0506 - acc: 0.9831

KeyboardInterrupt: 

In [41]:
score = model.evaluate(X_test, y_test, verbose=VERBOSE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 19s 2ms/sample - loss: 0.0406 - acc: 0.9869

Test score: 0.04056414132123464
Test accuracy: 0.9869


---

In [43]:
# import matplotlib.pyplot as plt

In [44]:
history_dict = history.history
history_dict.keys()

NameError: name 'history' is not defined

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf()   # clear figure
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

---

In [2]:
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.python.client import device_lib

# print('tensorflow:', tf.__version__)
# print('keras:', keras.__version__)

# if tf.test.is_built_with_cuda():
#     print('GPU devices:\n  ',
#         [ [x.name, x.physical_device_desc] 
#           for x in device_lib.list_local_devices() 
#           if x.device_type == 'GPU' ]
#     )
#     print('default GPU device:', tf.test.gpu_device_name() )

# else:
#     print('no GPU device found')
    

tensorflow: 1.13.1
keras: 2.2.4-tf
no GPU device found
